In [3]:
import numpy as np

class BinomialOptionPricing:
    def __init__(self, S, K, T, r, sigma, N, option_type='call', american=False):
        """
        S: Initial stock price
        K: Strike price
        T: Time to maturity (in years)
        r: Risk-free interest rate
        sigma: Volatility (standard deviation of stock's returns)
        N: Number of steps in the binomial tree
        option_type: 'call' or 'put'
        american: True for American options, False for European options
        """
        self.S = S
        self.K = K
        self.T = T
        self.r = r
        self.sigma = sigma
        self.N = N
        self.dt = T / N  # Time per step
        self.discount = np.exp(-r * self.dt)  # Discount factor
        self.u = np.exp(sigma * np.sqrt(self.dt))  # Up movement factor
        self.d = 1 / self.u  # Down movement factor
        self.p = (np.exp(r * self.dt) - self.d) / (self.u - self.d)  # Risk-neutral probability
        self.option_type = option_type
        self.american = american

    def price(self):
        stock_prices = np.zeros((self.N + 1, self.N + 1))
        stock_prices[0, 0] = self.S

        for i in range(1, self.N + 1):
            for j in range(i + 1):
                stock_prices[j, i] = self.S * (self.u ** (i - j)) * (self.d ** j)

        option_values = np.zeros((self.N + 1, self.N + 1))
        for j in range(self.N + 1):
            if self.option_type == 'call':
                option_values[j, self.N] = max(0, stock_prices[j, self.N] - self.K)
            elif self.option_type == 'put':
                option_values[j, self.N] = max(0, self.K - stock_prices[j, self.N])

        for i in range(self.N - 1, -1, -1):
            for j in range(i + 1):
                option_value_if_held = (self.p * option_values[j, i + 1] +
                                        (1 - self.p) * option_values[j + 1, i + 1]) * self.discount
                if self.american:
                    if self.option_type == 'call':
                        option_value_if_exercised = max(0, stock_prices[j, i] - self.K)
                    else:
                        option_value_if_exercised = max(0, self.K - stock_prices[j, i])
                    option_values[j, i] = max(option_value_if_held, option_value_if_exercised)
                else:
                    option_values[j, i] = option_value_if_held

        return option_values[0, 0]


#Example
S = 50  # Initial stock price
K = 52  # Strike price
T = 2    # Time to maturity (1 year)
r = 0.05  # Risk-free interest rate
sigma = 0.3  # Volatility (20%)
N = 2    # Number of steps in the binomial tree

# European Call Option
european_call = BinomialOptionPricing(S, K, T, r, sigma, N, option_type='call', american=False)
print("European Call Option Price:", european_call.price())

# European Put Option
european_put = BinomialOptionPricing(S, K, T, r, sigma, N, option_type='put', american=False)
print("European Put Option Price:", european_put.price())


# American Call Option
american_call = BinomialOptionPricing(S, K, T, r, sigma, N, option_type='call', american=True)
print("American Call Option Price:", american_call.price())

# American Put Option
american_put = BinomialOptionPricing(S, K, T, r, sigma, N, option_type='put', american=True)
print("American Put Option Price:", american_put.price())



European Call Option Price: 9.194162707336549
European Put Option Price: 6.245708445206439
American Call Option Price: 9.194162707336549
American Put Option Price: 7.428401902704835
